##Loading Libraries & Datasets

In [2]:
import pandas as pd
import numpy as np
from datetime import datetime
from datetime import timedelta
import math
import matplotlib.pyplot as plt
from datetime import datetime
import re
import random

In [3]:
item_metadata_filepath = './raw_data/item_metadata.csv'
submission_popular_filepath = './raw_data/submission_popular.csv'
train_filepath = './raw_data/train.csv'
test_filepath = './raw_data/test.csv'

In [4]:
submission_popular = pd.read_csv(submission_popular_filepath)
item_metadata = pd.read_csv(item_metadata_filepath)
train = pd.read_csv(train_filepath)
test = pd.read_csv(test_filepath)

**Unkown cells

In [ ]:
#loading train set
ReferenceTrain = pd.read_csv('/content/drive/My Drive/Trivago/Clean Dataset/train.csv')
ReferenceTrain = ReferenceTrain.drop(columns='Unnamed: 0')

#converting words values in referenc into NaN
ReferenceTrain.reference = ReferenceTrain.reference.apply(lambda x: pd.to_numeric(x, errors='coerce')).dropna().astype(int).apply(lambda x: str(x))

#dropping NaN in reference
ReferenceTrain = ReferenceTrain.drop(ReferenceTrain[ReferenceTrain.reference.isna()].index.tolist())

#what I meant by the previous step
train.reference, ReferenceTrain.reference

In [ ]:
ReferenceTimeSpent = (ReferenceTrain.groupby(['session_id', 'reference']).timestamp.max() 
                      - ReferenceTrain.groupby(['session_id', 'reference']).timestamp.min())
ReferenceTimeSpent

In [ ]:
ReferenceTrain.to_csv('train_ref.csv')                  #DO NOT RUN BEFORE INVESTIGATING FOR THE BEST PRACTICES
!cp train_ref.csv '/content/drive/My Drive/Trivago/Clean Dataset/'

##Feature Engineering

###Items Interacted With & Number of Interactions

Taking a sample of the training set to perform trials

In [ ]:
trainlet = train.iloc[0:10000]

#getting session_id with no clickouts
session_idNoClickouts = list(set(trainlet.session_id.unique()) - set(trainlet[trainlet.action_type == 'clickout item'].session_id.unique()))

#dropping sessions with no clickouts
trainlet = trainlet[~trainlet.session_id.isin(session_idNoClickouts)].drop(columns='Unnamed: 0')

#will be working on a grouped by sample, having the count of steps for each reference got interacted with
trainlet1 = trainlet.groupby(['session_id', 'reference'], sort=False).step.count().to_frame()

#converting an session_id as index into a column
trainlet1.reset_index(level=0, inplace=True)

#converting an reference as index into a column
trainlet1.reset_index(level=0, inplace=True)

#changing order of columns
trainlet1 = trainlet1[['session_id', 'reference','step']]

#takng a look
trainlet1.reference.head(10)

In [ ]:
#converting alphabetic values in reference attribute into NaN by converting all values into numeric, then again converting values into string
trainlet1.reference = trainlet1.reference.apply(lambda x: pd.to_numeric(x, errors='coerce')).dropna().astype(int).apply(lambda x: str(x))

#taking a look
trainlet1.reference.head(10)

In [ ]:
#for the sake of comparison creating another version of trainlet by repeating the previous steps
trainlet0 = trainlet.groupby(['session_id', 'reference'], sort=False).step.count().to_frame()
trainlet0.reset_index(level=0, inplace=True)
trainlet0.reset_index(level=0, inplace=True)
trainlet0 = trainlet0[['session_id', 'reference','step']]

#taking a look
trainlet0

In [ ]:
#changing the name of attribute step into NumberOfInteractions
trainlet0 = trainlet0.rename(columns = {'step':'NumberOfInteractions'})
trainlet1 = trainlet1.rename(columns = {'step':'NumberOfInteractions'})

#taking a look
trainlet1

In [ ]:
#for future use before dropping the NaN values, getting the indeces of the word values in reference attribute
#getting the index of words values in reference attribute
WordsInReferenceIndex = list(pd.isnull(trainlet1).any(1).nonzero())[0].tolist()

#getting reference with words dataframe
ReferenceWordsDF = trainlet0.iloc[WordsInReferenceIndex, :]  #for future**

In [ ]:
#dropping NaN values(rows)
trainlet1 = trainlet1.dropna()

#taking a look
trainlet1

In [ ]:
#getting reference and Number of interactions lists
InteractedWithItems = trainlet1.groupby('session_id', sort=False)['reference'].apply(list)
NumberOfInteractions = trainlet1.groupby('session_id', sort=False)['NumberOfInteractions'].apply(list)

#taking a look
InteractedWithItems, NumberOfInteractions

In [ ]:
#converting series into lists
InteractedWithItems = InteractedWithItems.tolist()
NumberOfInteractions = NumberOfInteractions.tolist()

Adding the two attributes to trainlet

In [ ]:
#getting a list of indeces InteractedWithItems and NumberOfInteractions should be left joined to
FinalClickoutIndex = trainlet[trainlet.action_type=='clickout item'].groupby('session_id').tail(1).index.tolist()

#creating InteractedWithItems dataframe
InteractedWithItemsDF = pd.DataFrame({'FinalClickoutIndex':FinalClickoutIndex, 'InteractedWitItems':InteractedWithItems,
                                      'NumberOfInteractions':NumberOfInteractions}).set_index('FinalClickoutIndex')

#left join on dataset
trainlet = trainlet.join(InteractedWithItemsDF)
trainlet

###Time Spent

Creating a time spent attribute for each reference within a session.

In [ ]:
#This should be a seperate function

#getting session_id with no clickouts
session_idNoClickouts = list(set(trainlet.session_id.unique()) - set(trainlet[trainlet.action_type == 'clickout item'].session_id.unique()))

#dropping sessions with no clickouts
trainlet = trainlet[~trainlet.session_id.isin(session_idNoClickouts)].drop(columns='Unnamed: 0')

#obtaining the seconds spent on a reference by subtracting the time started viewing the item till the time of last interaction with the item
time_spent = trainlet.groupby(['session_id', 'reference'], sort=False).timestamp.apply(lambda x:(x.max() - x.min())).to_frame()

#converting an session_id as index into a column
time_spent.reset_index(level=0, inplace=True)

#converting an reference as index into a column
time_spent.reset_index(level=0, inplace=True)

#changing order of columns
time_spent = time_spent[['session_id', 'reference','timestamp']]

#changing the timestamp into SecondsSpent
time_spent = time_spent.rename(columns = {'timestamp':'SecondsSpent'})

#converting alphabetic values in reference attribute into NaN by converting all values into numeric, then again converting values into string
time_spent.reference = time_spent.reference.apply(lambda x: pd.to_numeric(x, errors='coerce')).dropna().astype(int).apply(lambda x: str(x))

#dropping NaN values(rows)
time_spent = time_spent.dropna()

#getting time spent of interactions lists
SecondsSpent = time_spent.groupby('session_id', sort=False)['SecondsSpent'].apply(list)

#converting into list
SecondsSpent = SecondsSpent.tolist()

#getting a list of indeces SecondsSpent to be left joined
FinalClickoutIndex = trainlet[trainlet.action_type=='clickout item'].groupby('session_id').tail(1).index.tolist()

#creating Seconds dataframe
SecondsSpentDF = pd.DataFrame({'FinalClickoutIndex':FinalClickoutIndex, 'SecondsSpent':SecondsSpent}).set_index('FinalClickoutIndex')

#left join on dataset
trainlet = trainlet.join(SecondsSpentDF)
trainlet

In [ ]:
time_spent.groupby('session_id', sort=False)['SecondsSpent'].apply(list)

In [ ]:
SecondsSpent = time_spent.groupby('session_id', sort=False)['SecondsSpent'].apply(list)

In [ ]:
#creating a function
def SecondsSpent(dataset):
  #getting session_id with no clickouts
  session_idNoClickouts = list(set(dataset.session_id.unique()) - set(dataset[dataset.action_type == 'clickout item'].session_id.unique()))

  #dropping sessions with no clickouts
  dataset = dataset[~dataset.session_id.isin(session_idNoClickouts)].drop(columns='Unnamed: 0')

  #obtaining the seconds spent on a reference by subtracting the time started viewing the item till the time of last interaction with the item
  time_spent = dataset.groupby(['session_id', 'reference'], sort=False).timestamp.apply(lambda x:(x.max() - x.min())).to_frame()

  #converting an session_id as index into a column
  time_spent.reset_index(level=0, inplace=True)

  #converting an reference as index into a column
  time_spent.reset_index(level=0, inplace=True)

  #changing order of columns
  time_spent = time_spent[['session_id', 'reference','timestamp']]

  #changing the timestamp into SecondsSpent
  time_spent = time_spent.rename(columns = {'timestamp':'SecondsSpent'})

  #converting alphabetic values in reference attribute into NaN by converting all values into numeric, then again converting values into string
  time_spent.reference = time_spent.reference.apply(lambda x: pd.to_numeric(x, errors='coerce')).dropna().astype(int).apply(lambda x: str(x))

  #dropping NaN values(rows)
  time_spent = time_spent.dropna()

  #getting time spent of interactions lists
  SecondsSpent = time_spent.groupby('session_id', sort=False)['SecondsSpent'].apply(list)

  #converting into list
  SecondsSpent = SecondsSpent.tolist()

  #getting a list of indeces SecondsSpent to be left joined
  FinalClickoutIndex = dataset[dataset.action_type=='clickout item'].groupby('session_id').tail(1).index.tolist()

  #creating Seconds dataframe
  SecondsSpentDF = pd.DataFrame({'FinalClickoutIndex':FinalClickoutIndex, 'SecondsSpent':SecondsSpent}).set_index('FinalClickoutIndex')

  #left join on dataset
  dataset = dataset.join(SecondsSpentDF)
return dataset

In [ ]:
trainlet2 = train.iloc[0:10000]
trainlet2.head()

In [ ]:
SecondsSpent(trainlet2)

In [ ]:
pd.DataFrame({'FinalClickoutIndex':FinalClickoutIndex, 'SecondsSpent':SecondsSpent}).set_index('FinalClickoutIndex')

In [ ]:
trainlet[trainlet.action_type=='clickout item'].groupby('session_id').tail(1).index.tolist()

In [ ]:
#dropping NaN values(rows)
trainlet1 = trainlet1.dropna()

#taking a look
trainlet1

In [ ]:
#getting a list of values of time spent on each reference in a session
time_spent_values = time_spent.values

#getting the index of the time_spent_values in dataset
index = trainlet.groupby(['session_id', 'reference'], sort=False).tail(1).index

#creating a dataframe for a left join on train set
time_spent_df = pd.DataFrame({'index':index, 'seconds_spent':time_spent_values})
time_spent_df.head(2)

In [ ]:
#making the index column as the index for dataframe
time_spent_df = time_spent_df.set_index('index')

#left join to train set on index
train = train.join(time_spent_df)
train.head(15)

In [ ]:
#exporting dataframe to Google drive  DO NOT RUN THIS CELL UNLESS MODIFIED
train.to_csv('train.csv')

In [ ]:
#loading file                         DO NOT RUN THIS CELL UNLESS MODIFIED (SEARCH FOR BEST PRACTICES IN THIS CASE)   
train = pd.read_csv('/content/drive/My Drive/Trivago/Clean Dataset/train.csv')
train.head(15)

###Price

In [ ]:
Taking a session as an example, if there are three unique references. If we had the ones that have the same impressions which are the first two, and we get a table out of the impressions and prices, we will find that they have the same price exactly.
By proceeding with the KNN on this table(features can later on be added, and analysis on how different properties matter), we can have a list of 5 items at least(which are the most important) and go from there.	

In [ ]:
len(train[train.session_id == 'aff3928535f48'][train.action_type == 'clickout item'].tail(1).impressions.values[0].split('|'))

In [ ]:
items = train[train.session_id == 'aff3928535f48'][train.action_type == 'clickout item'].tail(1).impressions.values[0].split('|')

In [ ]:
all_interacted_with_items = train[train.session_id == 'aff3928535f48'].reference.unique().tolist()
interacted_with_items = []
interacted_with_items_prices = []
for item in all_interacted_with_items:
  for i in range(len(impression)):
    if item == impression[i]:
        interacted_with_items.append(item)
        print(item)
        interacted_with_items_prices.append(price[i])
        print(price[i])

In [ ]:
for item in all_interacted_with_items:
  for i in range(len(impression)):
    if item == impression[i]:
      print(item)

In [ ]:
all_interacted_with_items

In [ ]:
def impression_price(session_id):               #the isssue with this function is that ignores the first clickouts
                                                #(there are valuable infromation that can be extracted)
  try:                                          #some of the functions don't apply on the some sessions
    impression = train[train.session_id == session_id][train.action_type == 'clickout item'].tail(1).impressions.values[0].split('|')
    price = train[train.session_id == session_id][train.action_type == 'clickout item'].tail(1).prices.values[0].split('|')
    price = list(map(int, price))               #converting list of strings into integers
    clickout_item = train[train.session_id == session_id][train.action_type == 'clickout item'].tail(1).reference.values[0]
    all_interacted_with_items = train[train.session_id == session_id].reference.unique().tolist()
    for i in range(len(impression)):            #getting the clickout item
      if clickout_item == impression[i]:
        rank = i
    interacted_with_items = []
    interacted_with_items_prices = []
    for item in all_interacted_with_items:      #getting interacted with items
      for i in range(len(impression)):
        if item == impression[i]:
          interacted_with_items.append(item)
          interacted_with_items_prices.append(price[i])
    plt.figure(figsize=(10,8))
    plt.title('Impressions and Prices', fontsize=30)
    plt.xlabel('Impressions', fontsize=20)
    plt.ylabel('Price', fontsize=20)
    plt.xticks(rotation=90)
    plt.plot(impression, price, 'o')
    plt.plot(interacted_with_items, interacted_with_items_prices, 'o', color='red')
    plt.plot(clickout_item, price[rank] , 'o', color='black')
  except:
    pass

listOfSessions = random.choices(train.session_id.unique(), k=10)
for session_id in listOfSessions:
  impression_price(session_id)


TRY \ to have a the code running normally in a well organized shape

In [ ]:
listOfSessions = train.session_id.unique()[0:100]
for session_id in listOfSessions:
  impression_price(session_id)

Impressions shown on the graphs are put in a order of the rank provided by Trivago last list. After taking a quick look at the graphs (sample), I can see a pattern of having the black dot (clickout item) somewhere close to the red dots, at least not very far away. 
I'll need to validate that the features prices and ranks play an important role of user choice eventually.